In [50]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf


# Import our input dataset
charity_df = pd.read_csv('./Resources/charity_data.csv')
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [51]:
# Dropping variables that are neither feature nor the target
charity_df= charity_df.drop(columns=["EIN","NAME"])
charity_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [52]:
# Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

In [53]:
# Check the number of unique values in each column
charity_df[charity_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [54]:
# Check the application type unique value counts to see if binning is required
app_count= charity_df.APPLICATION_TYPE.value_counts()
app_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [55]:
# Determine which values to replace for bucketing
replace_app = list(app_count[app_count < 156].index)

# Replace in DataFrame
for application in replace_app:
    charity_df.APPLICATION_TYPE = charity_df.APPLICATION_TYPE.replace(application,"Other")


# Check to make sure binning was successful
charity_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [56]:
# Check the unique value counts to see if binning is required
class_count= charity_df.CLASSIFICATION.value_counts()
class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1820        1
C5200        1
C2600        1
C4120        1
C1248        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [57]:
# Determine which values to replace for bucketing
replace_class = list(class_count[class_count < 116].index)

# Replace in DataFrame
for classification in replace_class:
    charity_df.CLASSIFICATION = charity_df.CLASSIFICATION.replace(classification,"Other")


# Check to make sure binning was successful
charity_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other      887
C7000      777
C1700      287
C4000      194
C5000      116
Name: CLASSIFICATION, dtype: int64

In [58]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_df[charity_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [59]:
# Merge one-hot encoded features and drop the originals
charity_df = charity_df.merge(encode_df,left_index=True, right_index=True)
charity_df = charity_df.drop(charity_cat,1)
charity_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [60]:
# Split our preprocessed data into our features and target arrays
y = charity_df["IS_SUCCESSFUL"].values
X = charity_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [61]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [62]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3920      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6,381
Trainable params: 6,381
Non-trainable params: 0
_________________________________________________________________


In [63]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [64]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 967us/step - loss: 0.5669 - accuracy: 0.7210
Epoch 2/100
804/804 [==============================] - 1s 981us/step - loss: 0.5522 - accuracy: 0.7309
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5490 - accuracy: 0.7324
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5475 - accuracy: 0.7318
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5463 - accuracy: 0.7335
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5456 - accuracy: 0.7334
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5453 - accuracy: 0.7329
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5440 - accuracy: 0.7346
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5439 - accuracy: 0.7354
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5430 - ac

804/804 [==============================] - 1s 958us/step - loss: 0.5332 - accuracy: 0.7413
Epoch 81/100
804/804 [==============================] - 1s 996us/step - loss: 0.5331 - accuracy: 0.7419
Epoch 82/100
804/804 [==============================] - 1s 939us/step - loss: 0.5330 - accuracy: 0.7417
Epoch 83/100
804/804 [==============================] - 1s 923us/step - loss: 0.5328 - accuracy: 0.7415
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5332 - accuracy: 0.7413
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5328 - accuracy: 0.7417
Epoch 86/100
804/804 [==============================] - 1s 982us/step - loss: 0.5328 - accuracy: 0.74140s - l
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5329 - accuracy: 0.7411
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5325 - accuracy: 0.7413
Epoch 89/100
804/804 [==============================] - 1s 955us/step - loss: 0

In [65]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5583 - accuracy: 0.7255 - 242ms/epoch - 903us/step
Loss: 0.5583240985870361, Accuracy: 0.7254810333251953


In [66]:
# Rereun the model with 200 epochs to see if accuracy reaches 75%
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
804/804 [==============================] - 1s 958us/step - loss: 0.5321 - accuracy: 0.7426
Epoch 2/200
804/804 [==============================] - 1s 962us/step - loss: 0.5316 - accuracy: 0.7420
Epoch 3/200
804/804 [==============================] - 1s 912us/step - loss: 0.5322 - accuracy: 0.7427
Epoch 4/200
804/804 [==============================] - 1s 942us/step - loss: 0.5317 - accuracy: 0.7418
Epoch 5/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5321 - accuracy: 0.7417
Epoch 6/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5318 - accuracy: 0.7420
Epoch 7/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5313 - accuracy: 0.7423
Epoch 8/200
804/804 [==============================] - 1s 972us/step - loss: 0.5327 - accuracy: 0.7422
Epoch 9/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5313 - accuracy: 0.7420
Epoch 10/200
804/804 [==============================] - 1s 998us/step - loss: 0.5

804/804 [==============================] - 1s 973us/step - loss: 0.5302 - accuracy: 0.7428
Epoch 81/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5300 - accuracy: 0.7421
Epoch 82/200
804/804 [==============================] - 1s 955us/step - loss: 0.5300 - accuracy: 0.7429
Epoch 83/200
804/804 [==============================] - 1s 952us/step - loss: 0.5303 - accuracy: 0.7432
Epoch 84/200
804/804 [==============================] - 1s 956us/step - loss: 0.5313 - accuracy: 0.7425
Epoch 85/200
804/804 [==============================] - 1s 917us/step - loss: 0.5318 - accuracy: 0.7430
Epoch 86/200
804/804 [==============================] - 1s 981us/step - loss: 0.5300 - accuracy: 0.7421
Epoch 87/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5302 - accuracy: 0.7429
Epoch 88/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5297 - accuracy: 0.7427
Epoch 89/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5301 

804/804 [==============================] - 1s 931us/step - loss: 0.5292 - accuracy: 0.7429
Epoch 159/200
804/804 [==============================] - 1s 948us/step - loss: 0.5290 - accuracy: 0.7432
Epoch 160/200
804/804 [==============================] - 1s 956us/step - loss: 0.5292 - accuracy: 0.7427
Epoch 161/200
804/804 [==============================] - 1s 921us/step - loss: 0.5295 - accuracy: 0.7424
Epoch 162/200
804/804 [==============================] - 1s 956us/step - loss: 0.5290 - accuracy: 0.7423
Epoch 163/200
804/804 [==============================] - 1s 965us/step - loss: 0.5294 - accuracy: 0.7423
Epoch 164/200
804/804 [==============================] - 1s 946us/step - loss: 0.5322 - accuracy: 0.7424
Epoch 165/200
804/804 [==============================] - 1s 940us/step - loss: 0.5286 - accuracy: 0.7433
Epoch 166/200
804/804 [==============================] - 1s 960us/step - loss: 0.5288 - accuracy: 0.7431
Epoch 167/200
804/804 [==============================] - 1s 920us/ste

In [67]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5710 - accuracy: 0.7235 - 150ms/epoch - 560us/step
Loss: 0.5709995031356812, Accuracy: 0.723498523235321


In [68]:
# Rereun the model with 400 epochs to see if accuracy reaches 75%
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=400)

Epoch 1/400
804/804 [==============================] - 1s 956us/step - loss: 0.5295 - accuracy: 0.7426
Epoch 2/400
804/804 [==============================] - 1s 952us/step - loss: 0.5292 - accuracy: 0.7430
Epoch 3/400
804/804 [==============================] - 1s 969us/step - loss: 0.5282 - accuracy: 0.7426
Epoch 4/400
804/804 [==============================] - 1s 977us/step - loss: 0.5284 - accuracy: 0.7434
Epoch 5/400
804/804 [==============================] - 1s 963us/step - loss: 0.5281 - accuracy: 0.7432
Epoch 6/400
804/804 [==============================] - 1s 1ms/step - loss: 0.5291 - accuracy: 0.7430
Epoch 7/400
804/804 [==============================] - 1s 1ms/step - loss: 0.5295 - accuracy: 0.7435
Epoch 8/400
804/804 [==============================] - 1s 1ms/step - loss: 0.5307 - accuracy: 0.7424
Epoch 9/400
804/804 [==============================] - 1s 1ms/step - loss: 0.5282 - accuracy: 0.7430
Epoch 10/400
804/804 [==============================] - 1s 1ms/step - loss: 0.529

804/804 [==============================] - 1s 906us/step - loss: 0.5284 - accuracy: 0.7432
Epoch 80/400
804/804 [==============================] - 1s 902us/step - loss: 0.5280 - accuracy: 0.7431
Epoch 81/400
804/804 [==============================] - 1s 906us/step - loss: 0.5282 - accuracy: 0.7430
Epoch 82/400
804/804 [==============================] - 1s 916us/step - loss: 0.5279 - accuracy: 0.7428
Epoch 83/400
804/804 [==============================] - 1s 919us/step - loss: 0.5289 - accuracy: 0.7434
Epoch 84/400
804/804 [==============================] - 1s 909us/step - loss: 0.5287 - accuracy: 0.7432
Epoch 85/400
804/804 [==============================] - 1s 904us/step - loss: 0.5279 - accuracy: 0.7437
Epoch 86/400
804/804 [==============================] - 1s 908us/step - loss: 0.5296 - accuracy: 0.7434
Epoch 87/400
804/804 [==============================] - 1s 913us/step - loss: 0.5303 - accuracy: 0.7439
Epoch 88/400
804/804 [==============================] - 1s 911us/step - loss:

804/804 [==============================] - 1s 1ms/step - loss: 0.5276 - accuracy: 0.7431
Epoch 158/400
804/804 [==============================] - 1s 1ms/step - loss: 0.5280 - accuracy: 0.7425
Epoch 159/400
804/804 [==============================] - 1s 1ms/step - loss: 0.5282 - accuracy: 0.7441
Epoch 160/400
804/804 [==============================] - 1s 1ms/step - loss: 0.5279 - accuracy: 0.7430
Epoch 161/400
804/804 [==============================] - 1s 1ms/step - loss: 0.5281 - accuracy: 0.7435
Epoch 162/400
804/804 [==============================] - 1s 1ms/step - loss: 0.5278 - accuracy: 0.7431
Epoch 163/400
804/804 [==============================] - 1s 1ms/step - loss: 0.5324 - accuracy: 0.7430
Epoch 164/400
804/804 [==============================] - 1s 1ms/step - loss: 0.5277 - accuracy: 0.7432
Epoch 165/400
804/804 [==============================] - 1s 1ms/step - loss: 0.5273 - accuracy: 0.7437
Epoch 166/400
804/804 [==============================] - 1s 1ms/step - loss: 0.5286 - a

804/804 [==============================] - 1s 944us/step - loss: 0.5278 - accuracy: 0.7442
Epoch 236/400
804/804 [==============================] - 1s 955us/step - loss: 0.5274 - accuracy: 0.7428
Epoch 237/400
804/804 [==============================] - 1s 908us/step - loss: 0.5273 - accuracy: 0.7439
Epoch 238/400
804/804 [==============================] - 1s 960us/step - loss: 0.5271 - accuracy: 0.7436
Epoch 239/400
804/804 [==============================] - 1s 969us/step - loss: 0.5277 - accuracy: 0.7435
Epoch 240/400
804/804 [==============================] - 1s 919us/step - loss: 0.5273 - accuracy: 0.7436
Epoch 241/400
804/804 [==============================] - 1s 964us/step - loss: 0.5272 - accuracy: 0.7435
Epoch 242/400
804/804 [==============================] - 1s 959us/step - loss: 0.5274 - accuracy: 0.7437
Epoch 243/400
804/804 [==============================] - 1s 939us/step - loss: 0.5274 - accuracy: 0.7436
Epoch 244/400
804/804 [==============================] - 1s 951us/ste

804/804 [==============================] - 1s 946us/step - loss: 0.5266 - accuracy: 0.7439
Epoch 313/400
804/804 [==============================] - 1s 912us/step - loss: 0.5268 - accuracy: 0.7442
Epoch 314/400
804/804 [==============================] - 1s 967us/step - loss: 0.5269 - accuracy: 0.7438
Epoch 315/400
804/804 [==============================] - 1s 951us/step - loss: 0.5270 - accuracy: 0.7440
Epoch 316/400
804/804 [==============================] - 1s 914us/step - loss: 0.5269 - accuracy: 0.7435
Epoch 317/400
804/804 [==============================] - 1s 955us/step - loss: 0.5269 - accuracy: 0.7439
Epoch 318/400
804/804 [==============================] - 1s 949us/step - loss: 0.5289 - accuracy: 0.7438
Epoch 319/400
804/804 [==============================] - 1s 911us/step - loss: 0.5270 - accuracy: 0.7441
Epoch 320/400
804/804 [==============================] - 1s 965us/step - loss: 0.5269 - accuracy: 0.7440
Epoch 321/400
804/804 [==============================] - 1s 944us/ste

804/804 [==============================] - 1s 990us/step - loss: 0.5267 - accuracy: 0.7437
Epoch 391/400
804/804 [==============================] - 1s 983us/step - loss: 0.5325 - accuracy: 0.7436
Epoch 392/400
804/804 [==============================] - 1s 988us/step - loss: 0.5268 - accuracy: 0.7440
Epoch 393/400
804/804 [==============================] - 1s 997us/step - loss: 0.5264 - accuracy: 0.7439
Epoch 394/400
804/804 [==============================] - 1s 981us/step - loss: 0.5265 - accuracy: 0.7443
Epoch 395/400
804/804 [==============================] - 1s 990us/step - loss: 0.5264 - accuracy: 0.7439
Epoch 396/400
804/804 [==============================] - 1s 965us/step - loss: 0.5265 - accuracy: 0.7443
Epoch 397/400
804/804 [==============================] - 1s 980us/step - loss: 0.5267 - accuracy: 0.7436
Epoch 398/400
804/804 [==============================] - 1s 996us/step - loss: 0.5270 - accuracy: 0.7439
Epoch 399/400
804/804 [==============================] - 1s 986us/ste

In [69]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5943 - accuracy: 0.7258 - 153ms/epoch - 572us/step
Loss: 0.5943487882614136, Accuracy: 0.7258309125900269
